In [1]:
from ai_base import *
from collections import OrderedDict
from typing import List, Tuple
import numpy as np
def _getOneHot(fdata: List[List], word_dict: OrderedDict) -> np.array:
    D = len(fdata)
    if len(word_dict) is 0:
        for row in fdata:
            for word in row:
                if not word in word_dict:
                    word_dict[word] = 1
        word_dict[None] = 0
    else:
        word_dict = dict(zip(word_dict.keys(), [0 for _ in word_dict.values()]))
        for row in fdata:
            for word in row:
                if word in word_dict:
                    word_dict[word] = 1
                else:
                    word_dict[None] = 1
    word_vec = word_dict.keys()
    word_order = dict(zip(word_vec,range(len(word_vec))))
    oneHot = np.zeros((D,len(word_dict)), dtype=np.int8)
    for i,row in enumerate(fdata):
        for word in row:
            if word in word_order:
                oneHot[i][word_order[word]] = 1
            else:
                oneHot[i][word_order[None]] = 1
    return oneHot
def OneHot(trainData: List[List], testData: List[List], savePath="")-> Tuple[np.array, np.array]:
    wdict = OrderedDict()
    trainX = _getOneHot(trainData, wdict)
    testX = _getOneHot(testData, wdict)
    return trainX, testX

trainData = CSV2List2('trainDataclean.csv')
X, _ = OneHot(trainData, [], "-1")
Y = np.uint8(CSV2List2('trainLabel.txt'))
D = np.c_[X, Y]
K = 2
N = X.shape[0]
M = X.shape[1]
P_Y = np.zeros((K, 1))
D_parted = list(range(K))
for k in range(2):
    D_parted[k] = np.array(list(filter(lambda v: v[-1] == k, D)))
    P_Y[k] = len(D_parted[k]) / len(D)

In [4]:
M

62761

In [2]:
onehotPXY = CSV2List2("NB_onehotPXY.csv")
for j in range(2):
    for i in range(len(onehotPXY[j])):
        onehotPXY[j][i] = eval(onehotPXY[j][i])

In [3]:
def split(s, t, X, Y):
    piece = int(N / s)
    vaildX = []
    vaildY = []
    if t == s - 1:
        vaildX = X[t * piece:]
        vaildY = Y[t * piece:]
    else:
        vaildX = X[t * piece: (t+1) * piece]
        vaildY = Y[t * piece: (t+1) * piece]
    return vaildX, vaildY

In [5]:
def onehot_predict(X):
    _PY = deepcopy(P_Y)
    for k in range(K):
        for i in range(M):
            if X[i] == 1:
                ret = np.log(onehotPXY[k][i][X[i]])
                _PY[k] += ret
    return np.argmax(_PY)
import os
def onehot_predicts(Xs):
    return [onehot_predict(X) for X in Xs]

In [7]:
from copy import deepcopy
for s in range(5, 7):
    acc = 0
    for t in range(s):
        vaildX, vaildY = split(s, t, X, Y)
        predictY = np.zeros_like(vaildY)
        predictY = list_multiprocess(vaildX, onehot_predicts, 36)
        correct = 0
        for i in range(len(predictY)):
            if(predictY[i] == vaildY[i]):
                correct += 1
        acc += correct / len(predictY)
    print(s, acc / s)

Waiting for all subprocesses done...
All subprocesses done.
Waiting for all subprocesses done...
All subprocesses done.
Waiting for all subprocesses done...
All subprocesses done.
Waiting for all subprocesses done...
All subprocesses done.
Waiting for all subprocesses done...
All subprocesses done.
5 0.9190000000000002
Waiting for all subprocesses done...
All subprocesses done.
Waiting for all subprocesses done...
All subprocesses done.
Waiting for all subprocesses done...
All subprocesses done.
Waiting for all subprocesses done...
All subprocesses done.
Waiting for all subprocesses done...
All subprocesses done.
Waiting for all subprocesses done...
All subprocesses done.
6 0.9190054334987008
